    See 'Lyrics Data Collection', the notebook is split after pulling the list of artists, song names

# 1. Identify the data source, requirements

    Import libraries

In [29]:
import pandas as pd

# library helper
# run: importnb-install from Conda before using
from importnb import Notebook
with Notebook(): 
    import Utility

# custom helper class (from jupyter notebook)
helper = Utility.Helper()

from bs4 import BeautifulSoup
import requests

import re

from string import ascii_lowercase

from datetime import datetime

# progress bars for long running functions
from tqdm import tqdm

import numpy as np

Class 'Helper' v1.3 has been loaded


In [30]:
# reload changes in Jupyter notebooks
from importlib import reload
with Notebook(): __name__ == '__main__' and reload(Utility)

    Define constants and control variables

In [31]:
DATA_PATH = '../../data/'

LYRICS_URL = 'https://www.lyrics.com'

LYRICS_FILENAME = 'lyrics.csv'

In [32]:
artist_df = pd.read_csv(DATA_PATH + LYRICS_FILENAME)

artist_df.head()

,artist_name,artist_URL,song_title,song_URL,song_lyrics,song_runtime,song_year
0,A Band of Bees,artist/A-Band-of-Bees/526494,The Rip [On Track],/lyric/29617041/A+Band+of+Bees/The+Rip+%5BOn+T...,As she walks in the room Scented and tall Hesi...,00:04:37,NaN
1,A Band of Bees,artist/A-Band-of-Bees/526494,Left Foot Stepdown,/lyric/10339897/A+Band+of+Bees/Left+Foot+Stepdown,I tremble down if your love\nCrumbles down for...,00:04:06,NaN
2,A Band of Bees,artist/A-Band-of-Bees/526494,Horsemen,/lyric/13436377/A+Band+of+Bees/Horsemen,He was thinking like a madman He was jealous o...,00:03:28,NaN
3,A Band of Bees,artist/A-Band-of-Bees/526494,Chicken Payback (Madlib's Soul Distortion Voca...,/lyric/25986351/A+Band+of+Bees/Chicken+Payback...,Chicken Pay the chicken back back Pay the ch...,00:00:00,NaN
4,A Band of Bees,artist/A-Band-of-Bees/526494,These Are the Ghosts,/lyric/10328580/A+Band+of+Bees/These+Are+the+G...,You should think of a lesson As a weapon in lo...,00:03:08,NaN


In [33]:
print("Length of lyrics file: ", len(artist_df))

print("Song Lyrics N/A (error):", artist_df['song_lyrics'].isnull().sum())

sl_not_found = (artist_df['song_lyrics'] == 'Not found').sum()
print("Song Lyrics not found", sl_not_found)

print("Song Lyrics not scraped:", artist_df['song_lyrics'].isnull().sum() - sl_not_found)

print("Song Runtime not scraped:", (artist_df['song_runtime'] == 0).sum())

print("Song Year not scraped:", artist_df['song_year'].isnull().sum())

Length of lyrics file:  1123907
Song Lyrics N/A (error): 107197
Song Lyrics not found 61071
Song Lyrics not scraped: 46126
Song Runtime not scraped: 0
Song Year not scraped: 681539


In [34]:
# Cast Song Lyrics column to str using a dictionary:
# artist_df = artist_df.astype({'song_lyrics': 'str'})

# artist_df.dtypes

# 2. Establish connection, collect sample data, identify data format

In [35]:
# i = 0

# song_url = LYRICS_URL + artist_df['song_URL'][i]
# song_html = requests.get(song_url).text
# song_soup = BeautifulSoup(song_html, 'html.parser')

# song_runtime = song_soup.find('div', class_= re.compile('lyric-details')).find('dd', class_= re.compile('dd-margin')).findNext('dd').text

# print('Song runtime:', song_runtime)

# # The <pre> tag is inside <iframe>, so try to load it from iframe source URL:
# song_lyrics = song_soup.find('pre', id='lyric-body-text').text

# re.sub('\r', '', song_lyrics).split('\n')

In [36]:
artist_df.shape

(1123907, 7)

# 3. Create function for bulk collection, transformation

In [37]:
# #DEBUGGING

# print('song url: ', song_url)

# print('song_runtime: ', song_runtime)

# #print('song_lyrics: ', song_lyrics.text)

# artist_df[i-5:i+1]

In [38]:
# # look for index of next runtime == 0 (default for column)
# curr = artist_df.index[artist_df.loc[:, 'song_runtime'] == 0][0]

# interval_range = 118425
# stop = curr + interval_range

# print("Next iteration running from ", curr, " to ", stop)

In [39]:
# for i in tqdm(range(curr, stop)):
#     song_url = LYRICS_URL + artist_df['song_URL'][i]
#     song_html = requests.get(song_url).text
#     song_soup = BeautifulSoup(song_html, 'html.parser')
    
#     # unfortunately lyric details are not always present, nor is time
#     # we'll therefore save the runtime as 0:00 and then update it if present
#     artist_df.at[i, 'song_runtime'] = datetime.strptime('0:00', '%M:%S').time()
    
#     song_details = song_soup.find('div', class_= re.compile('lyric-details'))
    
#     if (song_details != None): 
#         # the song runtime is most consistently next to the clock icon <i class=far fa-clock">
#         song_clock = song_details.find('i', class_= re.compile('fa-clock'))

#         if (song_clock != None): 
#             song_runtime = song_clock.parent.find_next_sibling().text
            
#             song_runtime_mins = int(song_runtime.split(':')[-2])
            
#             if (song_runtime_mins >= 30):
#                 artist_df.at[i, 'song_runtime'] = -1
#                 continue

#             #save runtime, converted to datetime format, to dataframe
#             artist_df.at[i, 'song_runtime'] = datetime.strptime(song_runtime, '%M:%S').time()
        
#             # check for song year, beside clock
#             if (song_clock.parent.find_previous_sibling() != None):
#                 song_year = int(song_clock.parent.find_previous_sibling().text)
                
#                 #save song year
#                 artist_df.at[i, 'song_year'] = song_year
                
#     song_lyrics = song_soup.find('pre', id='lyric-body-text')

#     if (song_lyrics == None): 
#         artist_df.at[i, 'song_lyrics'] = 'Not found'
#     else:
#         # Set value at specified row/column pair, replacing '\n' with ' '
#         artist_df.at[i, 'song_lyrics'] = re.sub('\r\n', ' ', song_lyrics.text)

In [42]:
# second pass
indices = artist_df[artist_df['song_lyrics'].isnull()].index

length_indices = len(indices)

iteration_fraction = 1

In [44]:
# second pass

print("Before loop, Song Lyrics N/A (error):", artist_df['song_lyrics'].isnull().sum())

for i in tqdm(range(0, round(len(indices) / iteration_fraction))):
    
    index = indices[i]
    
    song_url = LYRICS_URL + artist_df.iloc[index]['song_URL']
    song_html = requests.get(song_url).text
    song_soup = BeautifulSoup(song_html, 'html.parser')
    
    song_lyrics = song_soup.find('pre', id='lyric-body-text')

    if (song_lyrics == None): 
        artist_df.at[index, 'song_lyrics'] = 'Not found'
    else:
        # Set value at specified row/column pair, replacing '\n' with ' '
        artist_df.at[index, 'song_lyrics'] = re.sub('\r\n', ' ', song_lyrics.text)
        
print("After loop, Song Lyrics N/A (error):", artist_df['song_lyrics'].isnull().sum())


Before loop, Song Lyrics N/A (error): 88316


100%|█████████████████████████████████████████████████████████████████████████| 88316/88316 [26:10:45<00:00,  1.07s/it]


After loop, Song Lyrics N/A (error): 0


# 4. Save data for later analysis

In [45]:
artist_df.to_csv(DATA_PATH + LYRICS_FILENAME, index=False)

In [40]:
# solve for UnicodeEncodeError: 'utf-8' codec can't encode characters in position 2046-2047: surrogates not allowed
artist_save = artist_df

artist_df['song_lyrics'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

0
1
2
3
4
